In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [35]:
from datasets import load_dataset

path = "/content/18GazeteHaberleri_2-3Ekim2019.csv"
reviews_dataset = load_dataset("csv", data_files= path)

In [37]:
reviews_dataset["train"]

Dataset({
    features: ['SIRA', 'GAZETE', 'BASLIK', 'URL', 'HABERMETNI'],
    num_rows: 2733
})

In [38]:
reviews_dataset['train'][:2]

{'SIRA': [1467, 1468],
 'GAZETE': ['Gazete Duvar', 'T24'],
 'BASLIK': ['Hentbol sezonu zorluklarla başladı',
  "50+1' baraj tartışmasına Cumhur İttifakı'nın küçük ortağı da katıldı: Sulandırmak olur\xa0"],
 'URL': ['https://www.gazeteduvar.com.tr/spor/2019/10/02/hentbol-sezonu-zorluklarla-basladi/',
  'https://t24.com.tr/haber/50-1-baraj-tartismasina-cumhur-ittifaki-nin-kucuk-ortagi-da-katildi-sulandirmak-olur,841964'],
 'HABERMETNI': ["Hentbol ligi aslında bu sezon pek çok zorluklar ile bir araya gelebildi. Fon üretilememesi, maliyetlerin artması, spor camiasından desteğin sınırlı alması, yönetim hataları nedeni ile iki yılda 57 kulüp hentboldan çekildi. Erkeklerde Selka gibi çok değerli bir yapı terk etti hentbolu. Batman ve Adıyaman Belediyeleri kapattı. Yozgat Bozok çıkmayı reddedip Başkent Akademi'ye devretti takımı. Bu kadar zorluklar ile uğraşan hentbol camiası, İstanbul Belediyesi'ne ait olan ve bulunduğu bölgede hentbol ölçülerine sahip tek tesisi, Ümraniye Haldun Alagaş Spor 

In [39]:
reviews_sample = reviews_dataset["train"].shuffle(seed=42).select(range(2733))

In [40]:
reviews_sample

Dataset({
    features: ['SIRA', 'GAZETE', 'BASLIK', 'URL', 'HABERMETNI'],
    num_rows: 2733
})

In [41]:
reviews_sample = reviews_sample.remove_columns('SIRA')
reviews_sample = reviews_sample.remove_columns('GAZETE')
reviews_sample = reviews_sample.remove_columns('BASLIK')
reviews_sample = reviews_sample.remove_columns('URL')
reviews_sample

Dataset({
    features: ['HABERMETNI'],
    num_rows: 2733
})

In [42]:
reviews_sample = reviews_sample.rename_column(
    original_column_name="HABERMETNI", new_column_name="review"
)
reviews_sample

Dataset({
    features: ['review'],
    num_rows: 2733
})

In [43]:
def compute_review_length(example):
    return {"review_length": len(example["review"].split())}

In [44]:
reviews_sample = reviews_sample.map(compute_review_length)
reviews_sample[0]

{'review': "Libya Ulusal Ordusu sözcüsü, Mitika ve Misrata havalimanlarında Türk yapımı İHA'lara füzeler eklendiğini iddia etti",
 'review_length': 15}

In [45]:
reviews_sample.sort("review_length")[:3]

{'review': ['Tehdit afet,gündem davet', 'Siyaset - .', 'Roni Batt  çiziyor.'],
 'review_length': [3, 3, 3]}

In [46]:
reviews_sample = reviews_sample.filter(lambda x: x["review_length"] > 10)
print(reviews_sample.num_rows)

Filter:   0%|          | 0/2733 [00:00<?, ? examples/s]

2575


In [47]:
reviews_sample[:3]

{'review': ["Libya Ulusal Ordusu sözcüsü, Mitika ve Misrata havalimanlarında Türk yapımı İHA'lara füzeler eklendiğini iddia etti",
  "Ataşehir meyve sebze halinde silah sesleri Ataşehir'de bulunan meyve sebze halinde alkollü olduğu ileri sürülen bir kişi, pompalı tüfekle etrafa ateş açtı. Olayda 2 kişi yaralanırken, şüpheli polis tarafından gözaltına alındı. Şüpheli gözaltına alındıktan sonra hal esnafı tarafından linç edilmek istendi. Polis havaya ateş ederek kalabalığı dağıttı. O anlar bir vatandaşın cep telefonu kamerasına yansıdı.",
  "Saygı Öztürk: Bulgusuz, saray raporu. Saygı Öztürk tüm yazıları ile Sozcu.com.tr'de"],
 'review_length': [15, 55, 11]}

In [48]:
import re
def remove_repeated(example):
    example["review"] = example["review"].replace('...', '')
    example["review"] = example["review"].replace(',"', '. ')
    example["review"] = example["review"].replace('!.', '.')
    example["review"] = example["review"].replace('!,', '. ')
    example["review"] = example["review"].replace('"', '')
    example["review"] = re.sub('([a-zA-Z0-9zığüşöçZİĞÜŞÖÇ]),([a-zA-Z0-9zığüşöçZİĞÜŞÖÇ])', '\\1. \\2', example["review"])

    return {"review": example["review"].replace('Devamını oku', '')}

In [49]:
reviews_sample = reviews_sample.map(remove_repeated)
reviews_sample[:3]

Map:   0%|          | 0/2575 [00:00<?, ? examples/s]

{'review': ["Libya Ulusal Ordusu sözcüsü, Mitika ve Misrata havalimanlarında Türk yapımı İHA'lara füzeler eklendiğini iddia etti",
  "Ataşehir meyve sebze halinde silah sesleri Ataşehir'de bulunan meyve sebze halinde alkollü olduğu ileri sürülen bir kişi, pompalı tüfekle etrafa ateş açtı. Olayda 2 kişi yaralanırken, şüpheli polis tarafından gözaltına alındı. Şüpheli gözaltına alındıktan sonra hal esnafı tarafından linç edilmek istendi. Polis havaya ateş ederek kalabalığı dağıttı. O anlar bir vatandaşın cep telefonu kamerasına yansıdı.",
  "Saygı Öztürk: Bulgusuz, saray raporu. Saygı Öztürk tüm yazıları ile Sozcu.com.tr'de"],
 'review_length': [15, 55, 11]}

In [50]:
reviews_sample = reviews_sample.train_test_split(train_size=0.9, seed=42)
reviews_sample["validation"] = reviews_sample.pop("test")

reviews_sample

DatasetDict({
    train: Dataset({
        features: ['review', 'review_length'],
        num_rows: 2317
    })
    validation: Dataset({
        features: ['review', 'review_length'],
        num_rows: 258
    })
})

In [51]:
for key in reviews_sample["train"][0]:
    print(f"{key.upper()}: {reviews_sample['train'][0][key]}")

REVIEW: Eski Yargıtay üyesi Mehmet Özkan, FETÖ üyeliğinden 6 yıl 3 ay hapis cezasına çarptırıldı.
REVIEW_LENGTH: 14


In [52]:
from transformers import AutoTokenizer

context_length = 40
pretrained_tokenizer = AutoTokenizer.from_pretrained("gpt2")


outputs = pretrained_tokenizer(
    reviews_sample["train"][:2]["review"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=False,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Input IDs length: 2
Input chunk lengths: [40, 40]


In [53]:
print("vocab_size: ", len(pretrained_tokenizer))

vocab_size:  50257


In [56]:
txt = "Ankara'da ipler gerildi."
tokens = pretrained_tokenizer(txt)['input_ids']
print(tokens)

[2025, 74, 3301, 1549, 64, 1312, 20053, 27602, 688, 72, 13]


In [57]:
converted = pretrained_tokenizer.convert_ids_to_tokens(tokens)
print(converted)

['An', 'k', 'ara', "'d", 'a', 'Ġi', 'pler', 'Ġger', 'ild', 'i', '.']


In [58]:
def get_training_corpus():
    batch_size = 1000
    return (
        reviews_sample["train"][i : i + batch_size]["review"]
        for i in range(0, len(reviews_sample["train"]), batch_size)
    )
training_corpus = get_training_corpus()

In [59]:
for reviews in get_training_corpus():
    print(len(reviews))

1000
1000
317


In [60]:
vocab_size = 52000
new_tokenizer = pretrained_tokenizer.train_new_from_iterator(training_corpus,vocab_size)

In [61]:
new_tokenizer.eos_token_id

0

In [62]:
new_tokenizer.vocab_size

25920

In [68]:
txt = "Ankara'da ipler gerildi."
tokens = new_tokenizer(txt)['input_ids']
print(tokens)

[1894, 330, 65, 21745, 16834, 14]


In [69]:
converted = new_tokenizer.convert_ids_to_tokens(tokens)
print(converted)

['Ankara', "'d", 'a', 'Ġipler', 'Ġgerildi', '.']


In [70]:
print(len(new_tokenizer.tokenize(txt)))
print(len(pretrained_tokenizer.tokenize(txt)))

6
11


In [71]:
path="./"
file_name="cagrii-turkishTokenizer"
new_tokenizer.save_pretrained(path+file_name)

('./cagrii-turkishTokenizer/tokenizer_config.json',
 './cagrii-turkishTokenizer/special_tokens_map.json',
 './cagrii-turkishTokenizer/vocab.json',
 './cagrii-turkishTokenizer/merges.txt',
 './cagrii-turkishTokenizer/added_tokens.json',
 './cagrii-turkishTokenizer/tokenizer.json')

In [72]:
loaded_tokenizer = AutoTokenizer.from_pretrained("./cagrii-turkishTokenizer")

In [74]:
txt = "Ankara'da ipler gerildi."
tokens = new_tokenizer(txt)['input_ids']
print("trained tokenizer:", tokens,"Tokenize:",len(new_tokenizer.tokenize(txt)))
tokens = loaded_tokenizer(txt)['input_ids']
print("loaded tokenizer:", tokens,"Tokenize:",len(loaded_tokenizer.tokenize(txt)))

trained tokenizer: [1894, 330, 65, 21745, 16834, 14] Tokenize: 6
loaded tokenizer: [1894, 330, 65, 21745, 16834, 14] Tokenize: 6
